# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 9 2023 8:30AM,256992,10,0086364954,ISDIN Corporation,Released
1,Feb 9 2023 8:30AM,256992,10,0086364952,ISDIN Corporation,Released
2,Feb 9 2023 8:30AM,256992,10,0086364953,ISDIN Corporation,Released
3,Feb 9 2023 8:30AM,256992,10,0086364972,ISDIN Corporation,Released
4,Feb 9 2023 8:30AM,256992,10,0086364973,ISDIN Corporation,Released
5,Feb 9 2023 8:30AM,256992,10,0086364974,ISDIN Corporation,Released
6,Feb 9 2023 8:30AM,256992,10,0086365000,ISDIN Corporation,Released
7,Feb 9 2023 8:30AM,256992,10,0086364999,ISDIN Corporation,Released
8,Feb 9 2023 8:30AM,256991,10,0086365215,ISDIN Corporation,Released
9,Feb 9 2023 8:30AM,256991,10,0086365221,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,257009,Executing,1
51,257009,Released,3
52,257010,Released,16
53,257011,Released,33
54,257012,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257008,NaN,NaN,20.0
257009,NaN,1.0,3.0
257010,NaN,NaN,16.0
257011,NaN,NaN,33.0
257012,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256826,46.0,0.0,66.0
256827,14.0,0.0,2.0
256875,0.0,0.0,1.0
256904,31.0,0.0,3.0
256908,3.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256826,46,0,66
256827,14,0,2
256875,0,0,1
256904,31,0,3
256908,3,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256826,46,0,66
1,256827,14,0,2
2,256875,0,0,1
3,256904,31,0,3
4,256908,3,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256826,46,,66
1,256827,14,,2
2,256875,,,1
3,256904,31,,3
4,256908,3,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation
8,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation
23,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation
28,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation
54,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd"
87,Feb 9 2023 8:19AM,257012,10,Emerginnova
89,Feb 9 2023 8:16AM,257010,10,ISDIN Corporation
105,Feb 9 2023 8:13AM,257009,20,"Exact-Rx, Inc."
109,Feb 9 2023 8:12AM,257008,19,"AdvaGen Pharma, Ltd"
129,Feb 9 2023 8:09AM,257007,20,"Granules Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation,,,26
1,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation,,,23
2,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation,,,2
3,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation,,,3
4,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd",,,33
5,Feb 9 2023 8:19AM,257012,10,Emerginnova,,,2
6,Feb 9 2023 8:16AM,257010,10,ISDIN Corporation,,,16
7,Feb 9 2023 8:13AM,257009,20,"Exact-Rx, Inc.",,1,3
8,Feb 9 2023 8:12AM,257008,19,"AdvaGen Pharma, Ltd",,,20
9,Feb 9 2023 8:09AM,257007,20,"Granules Pharmaceuticals, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation,26,,
1,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation,23,,
2,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation,2,,
3,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation,3,,
4,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd",33,,
5,Feb 9 2023 8:19AM,257012,10,Emerginnova,2,,
6,Feb 9 2023 8:16AM,257010,10,ISDIN Corporation,16,,
7,Feb 9 2023 8:13AM,257009,20,"Exact-Rx, Inc.",3,1,
8,Feb 9 2023 8:12AM,257008,19,"AdvaGen Pharma, Ltd",20,,
9,Feb 9 2023 8:09AM,257007,20,"Granules Pharmaceuticals, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation,26,,
1,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation,23,,
2,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation,2,,
3,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation,3,,
4,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd",33,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation,26.0,NaN,NaN
1,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation,23.0,NaN,NaN
2,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation,2.0,NaN,NaN
3,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation,3.0,NaN,NaN
4,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd",33.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 8:30AM,256992,10,ISDIN Corporation,26.0,0.0,0.0
1,Feb 9 2023 8:30AM,256991,10,ISDIN Corporation,23.0,0.0,0.0
2,Feb 9 2023 8:30AM,256982,10,ISDIN Corporation,2.0,0.0,0.0
3,Feb 9 2023 8:30AM,256993,10,ISDIN Corporation,3.0,0.0,0.0
4,Feb 9 2023 8:22AM,257011,19,"AdvaGen Pharma, Ltd",33.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4625324,149.0,0.0,5.0
12,257002,1.0,0.0,0.0
15,1027488,72.0,0.0,91.0
16,256974,1.0,0.0,0.0
19,3597734,69.0,1.0,0.0
20,770946,4.0,2.0,0.0
21,1027946,4.0,0.0,0.0
22,513991,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4625324,149.0,0.0,5.0
1,12,257002,1.0,0.0,0.0
2,15,1027488,72.0,0.0,91.0
3,16,256974,1.0,0.0,0.0
4,19,3597734,69.0,1.0,0.0
5,20,770946,4.0,2.0,0.0
6,21,1027946,4.0,0.0,0.0
7,22,513991,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,149.0,0.0,5.0
1,12,1.0,0.0,0.0
2,15,72.0,0.0,91.0
3,16,1.0,0.0,0.0
4,19,69.0,1.0,0.0
5,20,4.0,2.0,0.0
6,21,4.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,149.0
1,12,Released,1.0
2,15,Released,72.0
3,16,Released,1.0
4,19,Released,69.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,5.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
Released,149.0,1.0,72.0,1.0,69.0,4.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,Released,149.0,1.0,72.0,1.0,69.0,4.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,Released,149.0,1.0,72.0,1.0,69.0,4.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()